In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from fastai.io import *
from fastai.conv_learner import *
import os
from fastai.column_data import *

# Any results you write to the current directory are saved as output.

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = 'data/nietzsche/'


In [4]:
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600893


In [5]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


In [7]:
#adding 0 value for padding:
chars.insert(0, "\0")
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [8]:
#map from chars - indices and back
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [9]:
# idx converts all characters to their index
idx = [char_indices[c] for c in text]
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [10]:
 ''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

# Three char model

**create inputs**

In [ ]:
#a list of every 4th char
cs=3
c1_dat = [idx[i] for i in range(0, len(idx)-cs,cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs,cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs,cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs,cs)]

In [ ]:
#our inputs
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

In [ ]:
#our input
y = np.stack(c4_dat)

In [ ]:
#1st 4 inputs & outputs
x1[:4], x2[:4], x3[:4]

In [ ]:
y[:4]

In [ ]:
x1.shape, y.shape

**create and train model**

In [23]:
#size of hidden state
n_hidden = 256

In [24]:
#size of embedding matrix
n_fac = 42


In [ ]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        
        #layer operation from input to hidden
        self.l_lin = nn.Linear(n_fac,n_hidden)
        
        #layer operationg from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        #layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
    
    def forward(self, c1, c2,c3):
        in1 = F.relu(self.l_lin(self.e(c1)))
        in2 = F.relu(self.l_lin(self.e(c2)))
        in3 = F.relu(self.l_lin(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [ ]:
md = ColumnarModelData.from_arrays('.',[-1], np.stack([x1,x2,x3],axis=1),y,bs=512)

In [ ]:
m = Char3Model(vocab_size,n_fac).cuda()

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [ ]:
opt = optim.Adam(m.parameters(), 1e-2)

In [ ]:
fit(m,md,1,opt,F.nll_loss)

In [ ]:
set_lrs(opt, 0.001)

In [ ]:
fit(m,md,1,opt,F.nll_loss)

## Testing

In [ ]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [ ]:
get_next('car')

In [ ]:
get_next('see')

In [ ]:
get_next('pla')

# Our first RNN

Lets create the inputs:


In [11]:
#size of our unrolled RNN
cs=8

For each 0 -7 create a list of every 8th character with that starting point. WHich will make the 8 inputs to our model.

In [12]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx) -cs)]

Then create a list of the next charater in each of these series, which will be our labels:

In [13]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [14]:
xs = np.stack(c_in_dat, axis=0)

In [15]:
xs.shape

(600885, 8)

In [16]:
y = np.stack(c_out_dat)

In [17]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

This is the next character after each sequence. 

In [18]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

# Create and train model

In [19]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [20]:
md=ColumnarModelData.from_arrays('.',val_idx,xs,y,bs=512)

In [21]:
class CharLoopModel(nn.Module):
    #an RNN
    def __init__(self,vocab_size,n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size,n_fac)
        self.l_in = nn.Linear(n_fac,n_hidden)
        self.l_hidden = nn.Linear(n_hidden,n_hidden)
        self.l_out =nn.Linear(n_hidden,vocab_size)
        
    def forward(self,*cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs,n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h),dim=-1)

In [25]:
m = CharLoopModel(vocab_size,n_fac).cuda()
opt = optim.Adam(m.parameters(),1e-2)

In [26]:
fit(m, md, 1, opt, F.nll_loss)


epoch      trn_loss   val_loss                              
    0      2.030953   2.040313  



[array([2.04031])]

In [27]:
set_lrs(opt,0.001)

In [28]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.736717   1.739696  



[array([1.7397])]

In [35]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)


In [36]:
m = CharLoopConcatModel(vocab_size,n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [37]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [38]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.821145   1.799464  



[array([1.79946])]

In [39]:
set_lrs(opt, 1e-4)

In [40]:
fit(m, md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.722957   1.722552  



[array([1.72255])]

# Lets test the model

In [41]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [42]:
get_next('for thos')

'e'

In [44]:
get_next('part of ')

't'

In [45]:
get_next('shi')

'n'

In [46]:
get_next('tr')

'o'

# RNN with pytorch

In [47]:
class CharRNN(nn.Module):
    def __init__(self,vocab_size,n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac,n_hidden)
        self.l_out = nn.Linear(n_hidden,vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]),dim=-1)

In [49]:
m = CharRNN(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(),1e-3)

In [50]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [51]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [52]:
ht = V(torch.zeros(1,512,n_hidden))
outp,hn = m.rnn(t,ht)
outp.size(),hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [53]:
t = m(*V(xs)); t.size()

torch.Size([512, 85])

In [54]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.849165   1.844802  
    1      1.66784    1.669055                              
    2      1.584553   1.59724                               
    3      1.533005   1.553613                              



[array([1.55361])]

In [55]:
set_lrs(opt,1e-4)

In [56]:
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.458044   1.510942  
    1      1.464209   1.506052                              



[array([1.50605])]

### Lets test the model

In [75]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [76]:
get_next('I am very ')

'p'

In [77]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [78]:
get_next_n('for thos', 40)

'for those of the sense of the sense of the sense'

# Multi-output model

### setup

Lets take non-overlapping sets of characters

In [79]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1,cs)]

We create the exact same thing, offset by 1 as our labels

In [80]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs,cs)]

In [81]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [82]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [83]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [84]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [85]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [86]:
md = ColumnarModelData.from_arrays('.', val_idx,xs,ys,bs=512)

In [87]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn  = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [89]:
m = CharSeqRnn(vocab_size,n_fac).cuda()
opt = optim.Adam(m.parameters(),1e-3)

In [90]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [91]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [92]:
fit(m, md,4,opt,nll_loss_seq)

epoch      trn_loss   val_loss                              
    0      2.635968   2.43601   
    1      2.305967   2.208475                              
    2      2.144119   2.091926                              
    3      2.046234   2.015903                              



[array([2.0159])]

In [93]:
set_lrs(opt, 1e-4)

In [94]:
fit(m,md,1,opt,nll_loss_seq)

epoch      trn_loss   val_loss                              
    0      1.993502   1.998528  



[array([1.99853])]

### Identity init!

In [95]:
m = CharSeqRnn(vocab_size,n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [96]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [97]:
fit(m,md,4,opt,nll_loss_seq)

epoch      trn_loss   val_loss                              
    0      2.329707   2.145268  
    1      2.054588   1.99214                               
    2      1.953355   1.931806                              
    3      1.896553   1.895064                             



[array([1.89506])]

In [98]:
set_lrs(opt,1e-3)

In [99]:
fit(m,md,4,opt,nll_loss_seq)

epoch      trn_loss   val_loss                             
    0      1.801179   1.822663  
    1      1.787866   1.815379                              
    2      1.780794   1.811947                              
    3      1.772717   1.805266                              



[array([1.80527])]

# Stateful model

### Setup

We need to create the train and validation folders.

In [9]:
os.makedirs(TRN, exist_ok=True)
os.makedirs(VAL, exist_ok=True)

train_perc = .8
with open(f'{PATH}nietzsche.txt','r') as fp:
    lines = fp.readlines()
    text_len = len(lines)
    part_train = open(f'{TRN}nietzsche1.text','w')
    part_val = open(f'{VAL}nietzsche2.text','w')
    for ix,l in enumerate(lines):
        
        if ix/text_len<train_perc:
            part_train.write(l)
        else:
            part_val.write(l)
    part_train.close()
    part_val.close()

In [4]:

from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt


In [10]:
%ls {PATH}trn

nietzsche1.text


In [12]:
TEXT = data.Field(lower=True,tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=3)

#dataloader length, #of tokens, #
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(942, 55, 1, 482972)

In [13]:
TEXT.vocab.itos

['<unk>',
 '<pad>',
 ' ',
 'e',
 't',
 'i',
 'a',
 'o',
 'n',
 's',
 'r',
 'h',
 'l',
 'd',
 'c',
 'u',
 'f',
 'm',
 'p',
 'g',
 ',',
 'y',
 'w',
 'b',
 'v',
 '-',
 '.',
 '"',
 'k',
 'x',
 ';',
 ':',
 'q',
 'j',
 '!',
 '?',
 '(',
 ')',
 "'",
 'z',
 '1',
 '2',
 '=',
 '_',
 '3',
 '[',
 ']',
 '4',
 '5',
 '6',
 '8',
 '7',
 '9',
 '0',
 'ä']

In [14]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             ' ': 2,
             'e': 3,
             't': 4,
             'i': 5,
             'a': 6,
             'o': 7,
             'n': 8,
             's': 9,
             'r': 10,
             'h': 11,
             'l': 12,
             'd': 13,
             'c': 14,
             'u': 15,
             'f': 16,
             'm': 17,
             'p': 18,
             'g': 19,
             ',': 20,
             'y': 21,
             'w': 22,
             'b': 23,
             'v': 24,
             '-': 25,
             '.': 26,
             '"': 27,
             'k': 28,
             'x': 29,
             ';': 30,
             ':': 31,
             'q': 32,
             'j': 33,
             '!': 34,
             '?': 35,
             '(': 36,
             ')': 37,
             "'": 38,
             'z': 39,
             '1': 40,
             '2': 41,
             '=':

## RNN

In [18]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac,n_hidden)
        self.l_out = nn.Linear(n_hidden,vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
        
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [19]:
m = CharSeqStatefulRnn(md.nt, n_fac,512).cuda()
opt = optim.Adam(m.parameters(),1e-3)

In [20]:
fit(m,md,4,opt,F.nll_loss)

epoch      trn_loss   val_loss                               
    0      2.075584   2.013662  
    1      1.954677   1.907412                               
    2      1.898146   1.853205                               
    3      1.860191   1.832287                               



[array([1.83229])]

In [21]:
set_lrs(opt,1e-4)
fit(m,md,4,opt,F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.805082   1.789638  
    1      1.823645   1.788723                               
    2      1.805956   1.780166                               
    3      1.817407   1.773289                               



[array([1.77329])]

## RNN Loop

In [22]:
def RNNCell(input, hidden,w_ih, w_hh,b_ih,b_hh):
    return F.tanh(F.linear(input,w_ih,b_ih) + F.linear(hidden,w_hh,b_hh))

In [31]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self,vocab_size,n_fac,bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size,n_fac)
        self.rnn = nn.RNNCell(n_fac,n_hidden)
        self.l_out = nn.Linear(n_hidden,vocab_size)
        self.init_hidden(bs)
        
    def forward(self,cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs:
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp,dim=-1).view(-1,self.vocab_size)
        
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))
            

In [32]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(),1e-3)

In [33]:
fit(m,md,4,opt,F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.885576   1.846599  
    1      1.700836   1.704093                               
    2      1.617943   1.636365                               
    3      1.570475   1.597563                               



[array([1.59756])]

### Gated Recurrent Unit (GRU)

GRUs provide a gating mechanism in RNNs, which solve the vanishing gradient problem. It has an updated and reset gate, the update gate aids the model to how much of the past info needs to be passed to the future. The reset gate tells the model how much information it should forget. 

In [39]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size,n_fac,bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size,n_fac)
        self.rnn = nn.GRU(n_fac,n_hidden)
        self.l_out = nn.Linear(n_hidden,vocab_size)
        self.init_hidden(bs)
        
    def forward(self,cs):
        bs =cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1,self.vocab_size)
    
    def init_hidden(self,bs): self.h = V(torch.zeros(1,bs,n_hidden))
        

In [40]:
m = CharSeqStatefulGRU(md.nt,n_fac,512).cuda()
opt = optim.Adam(m.parameters(),1e-3)

In [41]:
fit(m,md,6,opt,F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.763068   1.736233  
    1      1.575307   1.583125                               
    2      1.506523   1.522552                               
    3      1.447065   1.494771                               
    4      1.406013   1.47491                                
    5      1.384559   1.461793                               



[array([1.46179])]

## Putting it all together: LSTM

In [42]:
from fastai import sgdr

n_hidden=512

In [49]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size,n_fac,bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size,n_fac)
        self.rnn = nn.LSTM(n_fac,n_hidden,nl,dropout=0.5)
        self.l_out = nn.Linear(n_hidden,vocab_size)
        self.init_hidden(bs)
        
    def forward(self,cs):
        bs =cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1,self.vocab_size)
    
    def init_hidden(self,bs): 
        self.h = (V(torch.zeros(self.nl,bs,n_hidden)),
                 V(torch.zeros(self.nl,bs,n_hidden)))
        

In [50]:
m = CharSeqStatefulLSTM(md.nt,n_fac,512,2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2,1e-5)

In [51]:
os.makedirs(f'{PATH}models',exist_ok=True)

In [52]:
fit(m,md,2,lo.opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.841674   1.746071  
    1      1.725941   1.66715                               



[array([1.66715])]

In [53]:
on_end = lambda sched, cycle: save_model(m,f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl),cycle_mult=2,on_cycle_end=on_end)]
fit(m,md,2**4-1,lo.opt,F.nll_loss,callbacks=cb)

epoch      trn_loss   val_loss                              
    0      1.551199   1.498228  
    1      1.600009   1.529073                              
    2      1.479532   1.444113                              
    3      1.619239   1.558921                              
    4      1.54224    1.488055                              
    5      1.452668   1.427435                              
    6      1.40387    1.390097                              
    7      1.610819   1.54093                               
    8      1.570096   1.51861                               
    9      1.535928   1.481205                              
    10     1.486857   1.45211                               
    11     1.438682   1.421916                              
    12     1.403049   1.391765                              
    13     1.365043   1.366508                              
    14     1.333518   1.352941                              



[array([1.35294])]

## Test

In [54]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [55]:
get_next('for thos')


'e'

In [ ]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [ ]:
print(get_next_n('for thos', 400))

In [ ]:
import torch
print(torch.__version__)